#LIB INSTALATION
!pip install yfinance
!pip install crewai
!pip install 'crewai[tools]'
!pip install langchain

In [6]:
import json
import os
from datetime import datetime
from crewai import Agent, Task
from langchain.tools import Tool

import yfinance as yf

In [7]:
def fetch_stock_price(ticket):
  stock = yf.download(ticket, start="2023-08-08", end="2024-08-08")
  return stock

yahoo_finance_tool = Tool(
  name = "Yahoo Finance Tool",
  description = "Fetch Stock prices for {ticket} from the last year about a specific stock from API",
  func = lambda ticket: fetch_stock_price(ticket)
)

In [8]:
response = yahoo_finance_tool.run("AAPL")

[*********************100%%**********************]  1 of 1 completed


In [9]:
print(response)

                  Open        High  ...   Adj Close     Volume
Date                                ...                       
2023-08-08  179.690002  180.270004  ...  178.643387   67823000
2023-08-09  180.869995  180.929993  ...  177.043747   60378500
2023-08-10  179.479996  180.750000  ...  176.825165   54686900
2023-08-11  177.320007  178.619995  ...  176.884857   51988100
2023-08-14  177.970001  179.690002  ...  178.546371   43675600
...                ...         ...  ...         ...        ...
2024-08-01  224.369995  224.479996  ...  218.107544   62501000
2024-08-02  219.149994  225.600006  ...  219.605804  105568600
2024-08-05  199.089996  213.500000  ...  209.028061  119548600
2024-08-06  205.300003  209.990005  ...  206.990402   69660500
2024-08-07  206.899994  213.639999  ...  209.577423   63516400

[252 rows x 6 columns]
